# Extraction des données associées video/audio/texte

Dans ce notebook, on fait une extraction des données issu d'un dataset brut :

Choix des données à partir du dossier MOSI RAW :
Nous sommes partis des données issues du lien suivant : https://www.kaggle.com/datasets/mathurinache/cmu-mosi  

Vidéo — raw/Video/Full/  

- Fichiers conservés :
	-   .mp4
- Pourquoi on utilise ces données :
    - Vidéos complètes, une par ID
    - Alignées parfaitement avec les transcriptions complètes
    - Nécessaires pour l’extraction des expressions faciales (OpenFace)
- Pourquoi on n’utilise pas les autres dossiers :
    - Video/Segmented/ contient 2199 segments mais pas de transcription segmentée correspondante → impossible d’aligner texte/audio/vidéo

Audio — raw/Audio/WAV_16000/Full/
- Fichiers conservés :
    - .wav (16 kHz)
- Pourquoi on utilise ces données :
    - Audio brut complet, aligné 1:1 avec les vidéos complètes
    - Qualité élevée, indispensable pour MFCC + jitter + shimmer + pitch (AED)
- Pourquoi on n’utilise pas les autres dossiers :
    - Audio/Segmented/ : 2199 segments sans transcripts alignés

Transcript — raw/Transcript/Full/
- Fichiers conservés :
    - .textonly
- Pourquoi on utilise ces données :
    - Transcriptions complètes, propres et alignées avec les vidéos/audio complets
    - Parfaites pour l’encodage BERT (TextEncoder)
- Pourquoi on n’utilise pas les autres dossiers :
    - Transcript/Segmented/ contient des .annotprocessed mais :
        - ils ne couvrent pas tous les segments
        - donc ils ne correspondent pas aux segments audio/vidéo
        - donc impossible de reconstituer proprement un échantillon multimodal

-   Seules 93 vidéos ont simultanément :
    - un fichier vidéo complet (.mp4)
    - un fichier audio complet (.wav)
    - une transcription complète (.textonly)

-   Ce sont les seules données parfaitement alignées dans les trois modalités.
-   En multimodal, chaque échantillon doit contenir texte + audio + vidéo, sans quoi il est inutilisable.



In [5]:
import os

def print_tree_dirs(startpath):
    for root, dirs, _ in os.walk(startpath):
        level = root.replace(startpath, "").count(os.sep)
        indent = " " * 4 * level
        print(f"{indent}{os.path.basename(root)}/")

print_tree_dirs("raw")


raw/
    Audio/
        WAV_11025/
        WAV_16000/
            COVAREP/
            Full/
            Segmented/
    Transcript/
        Full/
        Segmented/
    Video/
        Full/
        Segmented/


In [8]:
import os

root = "raw"

paths = {
    "Audio segmented": os.path.join(root, "Audio", "WAV_16000", "Segmented"),
    "Transcript segmented": os.path.join(root, "Transcript", "Segmented"),
    "Video segmented": os.path.join(root, "Video", "Segmented"),
}

for name, p in paths.items():
    if os.path.exists(p):
        files = os.listdir(p)
        print(f"\n{name} ({len(files)} fichiers) :")
        print(files[:10])
    else:
        print(f"{name} introuvable")



Audio segmented (2199 fichiers) :
['03bSnISJMiM_1.wav', '03bSnISJMiM_10.wav', '03bSnISJMiM_11.wav', '03bSnISJMiM_12.wav', '03bSnISJMiM_13.wav', '03bSnISJMiM_2.wav', '03bSnISJMiM_3.wav', '03bSnISJMiM_4.wav', '03bSnISJMiM_5.wav', '03bSnISJMiM_6.wav']

Transcript segmented (93 fichiers) :
['03bSnISJMiM.annotprocessed', '0h-zjBukYpk.annotprocessed', '1DmNV9C1hbY.annotprocessed', '1iG0909rllw.annotprocessed', '2iD-tVS8NPw.annotprocessed', '2WGyTLYerpo.annotprocessed', '5W7Z1C_fDaE.annotprocessed', '6Egk_28TtTM.annotprocessed', '6_0THN4chvY.annotprocessed', '73jzhE8R1TQ.annotprocessed']

Video segmented (2199 fichiers) :
['03bSnISJMiM_1.mp4', '03bSnISJMiM_10.mp4', '03bSnISJMiM_11.mp4', '03bSnISJMiM_12.mp4', '03bSnISJMiM_13.mp4', '03bSnISJMiM_2.mp4', '03bSnISJMiM_3.mp4', '03bSnISJMiM_4.mp4', '03bSnISJMiM_5.mp4', '03bSnISJMiM_6.mp4']


In [13]:
import os
import shutil
import json

# 🔹 Chemins vers tes dossiers d'origine
audio_folder = "raw/Audio/WAV_16000/Full"
video_folder = "raw/Video/Full"
transcript_folder = "raw/Transcript/Full"

# 🔹 Dossier de sortie
output_folder = "MOSI_full"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(os.path.join(output_folder, "audio"), exist_ok=True)
os.makedirs(os.path.join(output_folder, "video"), exist_ok=True)
os.makedirs(os.path.join(output_folder, "transcript"), exist_ok=True)

# 🔹 Lister les fichiers disponibles
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]
transcript_files = [f for f in os.listdir(transcript_folder) if f.endswith(".textonly")]

# 🔹 Extraire les IDs
audio_ids = set(f.split(".")[0] for f in audio_files)
video_ids = set(f.split(".")[0] for f in video_files)
transcript_ids = set(f.split(".")[0] for f in transcript_files)

# 🔹 Trouver les IDs communs
common_ids = transcript_ids & audio_ids & video_ids

print(f"Nombre d'IDs trouvés (transcript) : {len(transcript_ids)}")
print(f"IDs valides (présents dans les 3 modalités) : {len(common_ids)}\n")

# 🔹 Copier les fichiers correspondants
for id_ in common_ids:
    # Audio
    for f in audio_files:
        if f.startswith(id_):
            shutil.copy(os.path.join(audio_folder, f), os.path.join(output_folder, "audio", f))
    # Video
    for f in video_files:
        if f.startswith(id_):
            shutil.copy(os.path.join(video_folder, f), os.path.join(output_folder, "video", f))
    # Transcript
    for f in transcript_files:
        if f.startswith(id_):
            shutil.copy(os.path.join(transcript_folder, f), os.path.join(output_folder, "transcript", f))

# 🔹 Sauvegarder les IDs dans un fichier
with open(os.path.join(output_folder, "ids.txt"), "w") as f:
    for id_ in sorted(common_ids):
        f.write(id_ + "\n")

# 🔹 Créer un fichier metadata.json minimal
metadata = {
    "num_samples": len(common_ids),
    "audio_format": ".wav",
    "video_format": ".mp4",
    "transcript_format": ".textonly"
}
with open(os.path.join(output_folder, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=4)

print(f"🎉 Dataset MOSI_full construit avec succès !")
print(f"Dossier créé : {output_folder}/")
print(f"Fichiers disponibles : audio/, video/, transcript/, ids.txt, metadata.json")


Nombre d'IDs trouvés (transcript) : 93
IDs valides (présents dans les 3 modalités) : 93

🎉 Dataset MOSI_full construit avec succès !
Dossier créé : MOSI_full/
Fichiers disponibles : audio/, video/, transcript/, ids.txt, metadata.json
